In [108]:
%pip install tensorflow==2.5.0 scikit-learn optuna

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [109]:
import pandas as pd
import numpy as np

In [110]:
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [111]:
import tensorflow as tf

In [112]:
print(tf.__version__)
print(np.__version__)

2.18.0
1.26.4


In [113]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [114]:
def get_mlp_model(input_shape,hidden_layer_one,dropout_one,hidden_layer_two,dropout_two):
    model = Sequential()
    model.add(Dense(hidden_layer_one, activation='relu',  kernel_regularizer=l2(0.001), input_shape=input_shape))
    if dropout_one !=0:
        model.add(Dropout(dropout_one))
    model.add(Dense(hidden_layer_two, activation='relu', kernel_regularizer=l2(0.001)))
    if dropout_two !=0:
        model.add(Dropout(dropout_two))
    model.add(Dense(1, activation='linear'))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer="Adam")

    return model

In [115]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    rrse = np.sqrt(np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    mape = np.mean(np.abs((predicted - trues) / trues)) * 100
    r2 = max(0, 1 - np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    # Calculez les autres mesures d'erreur ici
    return rmse, corr, mae, rae, rrse, mape, r2

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import ModelCheckpoint

In [117]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

# We start by loading the pre-processed data

In [118]:
order_history_data=pd.read_csv('../data/final_data.csv')
order_history_data.head()

,Inflation,Month,Year,pmi,PartNo,Description,Quantity,Sales Price,Total Sales,Swaraj Engines Stock Price,M&M Stock Price,Escorts Kuboto Stock Price
0,7.59,1,2020,250,0503BA0290N-RK,ALFA HUB,85,2118.60,59367,1125.0,532.9,630.0
1,7.59,1,2020,250,0503BA0290N,Front Wheel Hub,318,3536.35,59367,1125.0,532.9,630.0
2,7.59,1,2020,250,0502BA1050N,Alfa Rear Wheel Hub LH,209,1383.36,59367,1125.0,532.9,630.0
3,7.59,1,2020,250,0502BA1060N,Alfa Rear Wheel Hub RH,119,4849.50,59367,1125.0,532.9,630.0
4,7.59,1,2020,250,0602AAB00260N,Brake Drum ALFA,134,2462.26,59367,1125.0,532.9,630.0


# Deep learning model optimization

In [119]:
X = order_history_data.copy()
y = order_history_data['Quantity']

X_encoded = pd.get_dummies(X, columns=['Description', 'PartNo'])

X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [120]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Validation Set:
MSE (Mean Squared Error) on the validation set: 9.827682932626185
R-squared (Coefficient of Determination) on the validation set: 0.9995434233230256

Test Set:
MSE (Mean Squared Error) on the test set: 13.485153652621674
R-squared (Coefficient of Determination) on the test set: 0.9993998228489208


Using Bayesian optimization to find the best hyperparameters

In [121]:
# # Objective function for Optuna optimization
# def objective(trial):
#     # Define the hyperparameters to be optimized
#     hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32])
#     hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [32])
#     dropout_one = trial.suggest_uniform('dropout_one', 0.2, 0.4)
#     dropout_two = trial.suggest_uniform('dropout_two', 0.1, 0.3)
#     # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
#     batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
#     epochs = trial.suggest_categorical('epochs', [200])



#     model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


#     # Define the ModelCheckpoint callback
#     checkpoint_filepath = 'model_checkpoint.h5'
#     model_checkpoint = ModelCheckpoint(
#         filepath=checkpoint_filepath,
#         save_best_only=True,
#         monitor='val_loss',
#         mode='min',
#         verbose=0
#     )

#     # Train the model on the training data with validation data and checkpoint callback
#     history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
#                         , validation_data=(X_val, y_val),
#                         callbacks=[model_checkpoint])

#     # Load the best weights from the saved checkpoint
#     model.load_weights(checkpoint_filepath)

#     # Evaluate the performance of the model on the test set
#     y_test_pred = model.predict(X_test)

#     # Calculate performance metrics
#     mse_test = mean_squared_error(y_test, y_test_pred)
#     # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

#     return mse_test

# # Configure Optuna to use the GPU for exhaustive searches
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=65, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# # Show best hyperparameters found
# print("Best hy    perparameters:")
# print(study.best_params)
# print("Best MSE:", study.best_value)

In [122]:
# best_params = study.best_params
# best_mse = study.best_value

In [123]:
# print(best_params)
# print(best_mse)

Using Grid Search

In [124]:
# r2=0

# for hidden_layer_one in [25,50,75,100,125,150,175,200,225,350,300,350,400]:
#   for hidden_layer_two in [325,50,75,100,125,150,175,200,225,350,300,350,400]:
#       for dropout_one in [0,0.1,0.2,0.3,0.4,0.5]:
#         for dropout_two in [0,0.1,0.2,0.3,0.4,0.5]:
#           # for batch_size in [32,64,128,256]:


#             model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

#             # Define the ModelCheckpoint callback
#             checkpoint_filepath = 'model_checkpoint.h5'
#             model_checkpoint = ModelCheckpoint(
#                 filepath=checkpoint_filepath,
#                 save_best_only=True,
#                 monitor='val_loss',
#                 mode='min',
#                 verbose=0
#             )

#             # Train the model on the training data with validation data and checkpoint callback
#             history = model.fit(X_train, y_train, epochs=300, batch_size=256,
#                                 verbose=0, validation_data=(X_val, y_val),
#                                 callbacks=[model_checkpoint])

#             # Load the best weights from the saved checkpoint
#             model.load_weights(checkpoint_filepath)

#             # Evaluate the performance of the model on the test set
#             y_test_pred = model.predict(X_test)

#             # Calculate performance metrics
#             mse_test = mean_squared_error(y_test, y_test_pred)
#             rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

#             if r2_2>r2:
#               r2=r2_2
#               print("{},{},{},{}".format(hidden_layer_one,hidden_layer_two,dropout_one,dropout_two) )
#               print("RMSE:", rmse2)
#               print("Corrélation:", corr2)
#               print("MAE:", mae2)
#               print("RAE:", rae2)
#               print("RRSE:", rrse2)
#               print("MAPE:", mape2)
#               print("R2:", r2_2)
#               print("MSE:", mse_test)
#               print("----------------------------------")

Training model with best hyperparamters

In [125]:
# best parameters

# hidden_layer_one=best_params['hidden_layer_one']
# hidden_layer_two=best_params['hidden_layer_two']
# dropout_one=best_params['dropout_one']  
# dropout_two=best_params['dropout_two']

# epochs=best_params['epochs']
# batch_size=best_params['batch_size']


hidden_layer_one=32
hidden_layer_two=32
dropout_one=0.20717484583420542
dropout_two=0.11283000231601609

epochs=200
batch_size=128


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = '../checkpoints/bayesian_final_model_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint, early_stopping])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Epoch 1/200


/home/panda/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 - 1s - 13ms/step - loss: 4037218.5000 - val_loss: 22566.2461
Epoch 2/200
57/57 - 0s - 2ms/step - loss: 780795.6250 - val_loss: 39983.3984
Epoch 3/200
57/57 - 0s - 3ms/step - loss: 335926.0625 - val_loss: 34999.2188
Epoch 4/200
57/57 - 0s - 2ms/step - loss: 221183.1406 - val_loss: 29573.6191
Epoch 5/200
57/57 - 0s - 2ms/step - loss: 121937.9062 - val_loss: 25034.7871
Epoch 6/200
57/57 - 0s - 2ms/step - loss: 56828.1875 - val_loss: 29352.3203
Epoch 7/200


57/57 - 0s - 3ms/step - loss: 39603.4883 - val_loss: 22121.8496
Epoch 8/200
57/57 - 0s - 2ms/step - loss: 33783.9648 - val_loss: 22183.2754
Epoch 9/200
57/57 - 0s - 3ms/step - loss: 30440.8594 - val_loss: 25702.2109
Epoch 10/200


57/57 - 0s - 2ms/step - loss: 28981.3418 - val_loss: 18755.6680
Epoch 11/200


...

57/57 - 0s - 2ms/step - loss: 11781.2539 - val_loss: 4274.0493
Epoch 23/200


57/57 - 0s - 2ms/step - loss: 13073.7881 - val_loss: 2337.3325
Epoch 24/200


57/57 - 0s - 3ms/step - loss: 11253.7510 - val_loss: 1718.0424
Epoch 25/200
57/57 - 0s - 2ms/step - loss: 11703.4854 - val_loss: 2603.6455
Epoch 26/200
57/57 - 0s - 2ms/step - loss: 11738.5449 - val_loss: 2240.3801
Epoch 27/200
57/57 - 0s - 2ms/step - loss: 10931.4297 - val_loss: 2070.8975
Epoch 28/200


57/57 - 0s - 2ms/step - loss: 10439.2891 - val_loss: 1090.3081
Epoch 29/200
57/57 - 0s - 2ms/step - loss: 9892.7637 - val_loss: 1609.3414
Epoch 30/200
57/57 - 0s - 2ms/step - loss: 10231.6016 - val_loss: 3249.4885
Epoch 31/200
57/57 - 0s - 2ms/step - loss: 10338.1162 - val_loss: 1670.1498
Epoch 32/200
57/57 - 0s - 2ms/step - loss: 9904.6133 - val_loss: 1589.4706
Epoch 33/200
57/57 - 0s - 2ms/step - loss: 9487.2852 - val_loss: 1277.5625
Epoch 34/200


57/57 - 0s - 2ms/step - loss: 9756.9189 - val_loss: 653.1475
Epoch 35/200
57/57 - 0s - 2ms/step - loss: 9585.3730 - val_loss: 2426.2754
Epoch 36/200
57/57 - 0s - 2ms/step - loss: 8958.3242 - val_loss: 1737.5212
Epoch 37/200
57/57 - 0s - 2ms/step - loss: 8897.8994 - val_loss: 1614.1183
Epoch 38/200
57/57 - 0s - 3ms/step - loss: 8711.7490 - val_loss: 2386.8318
Epoch 39/200
57/57 - 0s - 2ms/step - loss: 8645.0059 - val_loss: 2409.7065
Epoch 40/200
57/57 - 0s - 2ms/step - loss: 9577.5586 - val_loss: 2586.5203
Epoch 41/200
57/57 - 0s - 2ms/step - loss: 9256.9512 - val_loss: 1873.6342
Epoch 42/200
57/57 - 0s - 3ms/step - loss: 9005.3906 - val_loss: 1379.8479
Epoch 43/200
57/57 - 0s - 3ms/step - loss: 9312.2949 - val_loss: 892.0172
Epoch 44/200
57/57 - 0s - 2ms/step - loss: 8794.7607 - val_loss: 1818.4382
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step
RMSE: 25.863664111189173
Corrélation: 0.9957744442299397
MAE: 20.381084843685752
RAE: 0.16525361307616906
RRSE: 0.17254478222739958
MAPE: 17.89124432

Save the weights of the best model for later loading

In [126]:
# Save the best model in checkpoint only if it is better
model.save_weights('../checkpoints/bayesian_final.weights.h5')